In [1]:
require 'hdf5'
require 'rnn'


In [2]:
N = 2
myFile = hdf5.open('../data_preprocessed/'..tostring(N)..'-grams.hdf5','r')
data = myFile:all()
F_train = data['F_train']
input_data_valid = data['input_data_valid']
output_matrix_train = data['output_matrix_train']
input_matrix_train = data['input_matrix_train']
input_data_train = data['input_data_train']
input_data_valid_nospace = data['input_data_valid_nospace']
input_data_test = data['input_data_test']
myFile:close()

In [3]:
print(input_matrix_train:size())
print(output_matrix_train:size())
print(input_data_train:size())

 599903
      1
[torch.LongStorage of size 2]

 599903
[torch.LongStorage of size 1]

 599905
[torch.LongStorage of size 1]



In [4]:
-- TOFIX: outpout data does not contain the prediction for the last to the end char (here 7)
input_data_train:narrow(1,input_data_train:size(1)-10,11)

  1
 30
  1
 20
 12
 11
 11
 12
  9
  7
  2
[torch.LongTensor of size 11]



In [5]:
output_matrix_train:narrow(1,output_matrix_train:size(1)-10,11)

 2
 1
 2
 1
 2
 2
 2
 2
 2
 2
 2
[torch.DoubleTensor of size 11]



In [3]:
-- Formating the input
-- Currently using a hack to have nive divisions
n = input_data_train:size(1)
n_new = n - 905
len = 100
batch_size = 10
print(n_new)
print(n_new/(batch_size*len))

599000	
599	


In [4]:
-- Issue with last sequence if batch_size does not divide n
t_input = torch.split(input_data_train:narrow(1,1,n_new):view(batch_size,n_new/batch_size),len, 2)
t_output = torch.split(output_matrix_train:narrow(1,1,n_new):view(batch_size,n_new/batch_size),len, 2)

In [8]:
t_input[1]:size()

  10
 100
[torch.LongStorage of size 2]



In [5]:
function build_rnn(embed_dim, vocab_size, len)
    local batchRNN
    local params
    local grad_params
    -- generic RNN transduced
    batchRNN = nn.Sequential()
        :add(nn.LookupTable(vocab_size, embed_dim))
        :add(nn.SplitTable(1, batch_size))
    
    batchRNN:add(nn.Sequencer(nn.Recurrent(
       embed_dim, nn.Linear(embed_dim, embed_dim), 
       nn.Linear(embed_dim, embed_dim), nn.Tanh(), len)))
    -- Output
    batchRNN:add(nn.Sequencer(nn.Linear(embed_dim, 2)))
    batchRNN:add(nn.Sequencer(nn.LogSoftMax()))
    batchRNN:remember('both')

    -- Retrieve parameters (To do only once!!!)
    params, grad_params = batchRNN:getParameters()
    
    return batchRNN, params, grad_params
end

# Training

In [131]:
-- we can then use a simple BCE Criterion for backprop
pred = batchRNN:forward(t_inputT)
crit = nn.SequencerCriterion(nn.ClassNLLCriterion())
loss = crit:forward(pred, t_output_table)

dLdPred = crit:backward(pred, t_output_table)
batchRNN:backward(t_inputT, dLdPred)

In [6]:
function train_model(t_input, t_output, model, params, grad_params,
                     criterion, eta, nEpochs, batch_size, batch_number, len)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01
    local timer
    local pred
    local loss
    local dLdPred
    local t_inputT = torch.DoubleTensor(len,batch_size)
    local t_output_table

    -- To store the loss
    local av_L = 0
    
    -- Initializing all the parameters between -0.05 and 0.05
    for k=1,params:size(1) do
        params[k] = torch.uniform(-0.05,0.05)
    end

    for i = 1, nEpochs do
        -- timing the epoch
        timer = torch.Timer()
        av_L = 0
        
        -- mini batch loop
        for k = 1, batch_number do
            -- Mini batch data
                
            t_inputT:copy(t_input[k]:t())
            t_output_table = torch.split(t_output[k],1,2)
            --format the output
            for j=1,len do
                t_output_table[j] = t_output_table[i]:squeeze()
            end 
            
            -- reset gradients
            model:zeroGradParameters()
            pred = model:forward(t_inputT)
            loss = crit:forward(pred, t_output_table)
            av_L = av_L + loss

            dLdPred = crit:backward(pred, t_output_table)
            model:backward(t_inputT, dLdPred)
            
            -- gradient normalization with max norm 5 (l2 norm)
            grad_params:view(grad_params:size(1),1):renorm(1,2,5)
            model:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        print('Average Loss: '..av_L/math.floor(batch_number))
       
    end
end

In [7]:
vocab_size = 49
embed_dim = 16
len = 100
eta = 0.1
nEpochs = 1
batch_number = n_new/(batch_size*len)

-- Building model
batchRNN, params, grad_params = build_rnn(embed_dim, vocab_size, len)
crit = nn.SequencerCriterion(nn.ClassNLLCriterion())

In [8]:
train_model(t_input, t_output, batchRNN, params, grad_params,
                     criterion, eta, 5, batch_size, batch_number, len)

Epoch 1: 10.283056020737	
Average Loss: 47.798446118375	


Epoch 2: 10.49356508255	
Average Loss: 47.901252699071	


Epoch 3: 12.212630987167	
Average Loss: 46.838004103291	


Epoch 4: 12.556469917297	
Average Loss: 49.758562062299	


Epoch 5: 11.571686029434	
Average Loss: 48.248888095988	

